Alex Hemmerlin
Comp 435 Project 1
Data Exploration
English Handwritten Characters: https://www.kaggle.com/datasets/dhruvildave/english-handwritten-characters-dataset

About the Dataset:
The dataset contains 3,410 unique images of handwritten English characters. The characters are lowercase alphabetical: a-z, uppercase alphabetical: A-Z, and numeric digits: 0-9. This results in 62 classes (26+26+10) with 55 images of each class represented in the dataset.

The class label is the character that is represented in the image (0-9, A-Z, a-z). The feature is the image which is a 1200x900 png image.

In [114]:
# ***MY CODE*** Denotes code that I have written
# ***[WHERE THE CODE CAME FROM]*** Denotes code that I copied from the place specified
# ***[WHERE THE CODE CAME FORM] MODIFIED BY ME*** Denotes code that was copied and modified

In [115]:
# @InProceedings{deCampos09,
#   author    = "de Campos, T.~E. and Babu, B.~R. and Varma, M.",
#   title     = "Character recognition in natural images",
#   booktitle = "Proceedings of the International Conference on Computer
#   Vision Theory and Applications, Lisbon, Portugal",
#   month     = "February",
#   year      = "2009",
# }

In [ ]:
# TODO: Ideas for data exploration:
# 1. Write code to lower the resolution of the images
# 2. Write code to map the labels to integer values
# 3. Write code to plot a few random images with labels

In [ ]:
# ***MY CODE***
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

DATASET_PATH: str = 'dataset/english.csv'   # path to the csv file
BASE_IMAGE_PATH: str = 'dataset/'           # path to the Img folder (concat the image string in the csv to the end to get the path to the image)

# TODO: Change to False to not display some example images (they are large and take up a lot of screen space)
PRINT_EXAMPLES: bool = False

In [ ]:
# ***MY CODE***
df = pd.read_csv(DATASET_PATH)
print(df.head())


                image label
0  Img/img001-001.png     0
1  Img/img001-002.png     0
2  Img/img001-003.png     0
3  Img/img001-004.png     0
4  Img/img001-005.png     0


In [119]:
# ***MY CODE***
df["label"].value_counts()

0    55
k    55
X    55
Y    55
Z    55
     ..
P    55
Q    55
R    55
S    55
z    55
Name: label, Length: 62, dtype: int64

In [ ]:
# ***MY CODE***
if PRINT_EXAMPLES:
    display(Image.open(f'{BASE_IMAGE_PATH}{df["image"][0]}'))
    display(Image.open(f'{BASE_IMAGE_PATH}{df["image"][550]}'))
    display(Image.open(f'{BASE_IMAGE_PATH}{df["image"][1980]}'))